In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import bitsandbytes as bnb

# Install necessary libraries in Colab
!pip install --upgrade bitsandbytes transformers accelerate

# Define your Hugging Face token
hf_token = "your_new_hugging_face_token_here"  # Replace with your actual Hugging Face token

# Load LLaMA with bitsandbytes for quantization
model_name = "meta-llama/Meta-Llama-Guard-2-8B"  # Replace with the actual LLaMA model
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)

# Configure quantization with CPU offloading
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    load_in_8bit_fp32_cpu_offload=True  # Enable offloading to CPU for layers that don't fit in GPU memory
)

# Define a custom device map
device_map = {
    "transformer.h.0": 0,  # Load the first transformer layer on GPU
    "transformer.h.1": 0,  # Load the second transformer layer on GPU
    "transformer.h.2": "cpu",  # Offload additional layers to CPU if needed
    "transformer.lm_head": "cpu",  # Offload the final layer to CPU
}

# Load the model with quantization and offloading
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map=device_map,  # Use the custom device map
    token=hf_token
)

# Function to scrape content from a URL
def scrape_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup.get_text()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return ""

# Function to generate an answer using the LLaMA model
def generate_answer(prompt, scraped_content):
    combined_content = f"{prompt}\n\n{scraped_content}"

    # Tokenize input
    inputs = tokenizer(combined_content, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate text with the model
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
        )

    # Decode and return the output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Function to save the generated text to a text file
def save_to_text_file(generated_text, filename='questions.txt'):
    with open(filename, 'w', encoding='utf-8') as textfile:
        textfile.write(generated_text)

# Main function to process input and generate the output
def main(prompt, urls):
    scraped_content = ""
    for url in urls:
        print(f"Scraping content from {url}...")
        scraped_content += scrape_content(url) + "\n\n"

    if not scraped_content.strip():
        print("No content scraped from the provided URLs.")
        return

    for i in range(200):
        try:
            generated_text = generate_answer(prompt, scraped_content)
            if "No content generated" in generated_text:
                raise ValueError("Content generation failed.")
            print(f"Generated Answer for iteration {i + 1}:")
            print(generated_text)
            save_to_text_file(generated_text, filename=f'questions_{i + 1}.txt')
        except Exception as e:
            print(f"Skipping iteration {i + 1} due to error: {e}")

if __name__ == "__main__":
    # Example usage
    input_prompt = """Prompt for generating MCQs with LLaMA
    You are a cybersecurity expert specializing in cyber threat intelligence. Given the text below, please
    generate a maximum of 5 multiple-choice questions with four possible options each.
    Follow these requirements:
    1. Question Format: Each question must have four options. The options should be challenging and require
    careful consideration. Avoid creating options that could be interpreted as correct under different circumstances.
    2. Target Audience: The questions should be suitable for security professionals with three to five years of
    experience in cyber threat intelligence. Avoid generic questions such as “What is the objective?”, “Which
    operating system can be targeted?”.
    3. Content Coverage: Aim to cover various sections of the document to ensure a comprehensive evaluation of
    the candidate’s knowledge. Include context-specific questions that require an understanding of the document’s
    content.
    4. Technical Precision: Use precise terminology and concepts relevant to cyber threat intelligence. Incorporate
    situational or scenario-based questions where applicable.
    5. Include Technique IDs and Names: Ensure that all questions, where applicable, mention both the ID and the
    full name of the MITRE ATT&CK pattern technique.
    6. Premise Inclusion: Each question should include a premise indicating it pertains to MITRE ATT&CK,
    specifying the relevant platform (Enterprise, ICS, or Mobile) where necessary.
    7. Output Format: Return the output in text format with these columns:
    Question, Option A, Option B, Option C, Option D, Correct Answer (A, B, C, D), and Explanation.
    Important: Only return the text content """

    input_urls = [
        "https://www.nist.gov/cyberframework",
    ]

    main(input_prompt, input_urls)


Mounted at /content/drive
Skipping CVE CVE-2024-38080 due to errors with all API keys.
Skipping CVE CVE-2024-38112 due to errors with all API keys.
Skipping CVE CVE-2024-38021 due to errors with all API keys.
Generated CVSS Vector for CVE-2024-38053 using API key 1: **CVSS v3.1 Vector String:** CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:H/A:H
Generated CVSS Vector for CVE-2024-38053 using API key 4: **CVSS v3.1 Vector String:** CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:N/I:N/A:H
Generated CVSS Vector for CVE-2024-38053 using API key 7: CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H
Generated CVSS Vector for CVE-2024-38053 using API key 8: **CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H**
Skipping CVE CVE-2024-38080 due to errors with all API keys.
Skipping CVE CVE-2024-38112 due to errors with all API keys.
Skipping CVE CVE-2024-38021 due to errors with all API keys.
Skipping CVE CVE-2024-38053 due to errors with all API keys.
Skipping CVE CVE-2024-38053 due to errors with all API keys.
Skipping CVE

Error during API call for CVE-2024-38053 with API key 6, attempt 3: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error during API call for CVE-2024-38053 with API key 7, attempt 1: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error during API call for CVE-2024-38053 with API key 7, attempt 2: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error during API call for CVE-2024-38053 with API key 7, attempt 3: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


...

Error during API call for CVE-2024-38021 with API key 5, attempt 2: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error during API call for CVE-2024-38021 with API key 5, attempt 3: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error during API call for CVE-2024-38021 with API key 6, attempt 1: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error during API call for CVE-2024-38021 with API key 6, attempt 2: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error during API call for CVE-2024-38021 with API key 6, attempt 3: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.0-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Error during API call for CVE-2024-38021 with API key 7, attempt 1: HTTPConnectionPool(host='localhost', port=34229): Read timed out. (read timeout=600.0)
Error during API call for CVE-2024-38021 with API key 7, attempt 2: HTTPConnectionPool(host='localhost', port=34229): Read timed out. (read timeout=600.0)
Error during API call for CVE-2024-38021 with API key 7, attempt 3: HTTPConnectionPool(host='localhost', port=34229): Read timed out. (read timeout=600.0)
Error during API call for CVE-2024-38021 with API key 8, attempt 1: HTTPConnectionPool(host='localhost', port=34229): Read timed out. (read timeout=600.0)
Error during API call for CVE-2024-38021 with API key 8, attempt 2: HTTPConnectionPool(host='localhost', port=34229): Read timed out.